# Imports

In [1]:
import xgboost as xgb
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Train

In [19]:
train_set = pd.read_csv('train/train_encoded.csv')
train_set.head(10)

,len_location,len_keyword,len_text,len_text_original,diff_len_text,keyword_value_sin_stemming,keyword_value_con_stemming,keyword_value_mult,keyword_value_diff,keyword_sum,...,text_norm_value_2_gram,text_value_3_gram,text_norm_value_3_gram,text_value_4_gram,text_norm_value_4_gram,text_value_mult,text_value_diff,text_norm_value_mult,text_norm_value_diff,target
0,0,0,43,69,26,3.267254,0.03008,0.098278,3.237175,1.0,...,0.039129,0.0,0.0,0.0,0.0,6.328488,27.200895,1.034131,12.431300,1
1,0,0,36,38,2,3.267254,0.03008,0.098278,3.237175,1.0,...,0.039828,0.0,0.0,0.0,0.0,2.183374,14.948923,0.482472,7.746799,1
2,0,0,72,133,61,3.267254,0.03008,0.098278,3.237175,1.0,...,0.058736,0.0,0.0,0.0,0.0,15.902513,50.867533,2.458198,21.204076,1
3,0,0,50,65,15,3.267254,0.03008,0.098278,3.237175,1.0,...,0.040829,0.0,0.0,0.0,0.0,4.867260,24.192476,0.804374,11.435785,1
4,0,0,52,88,36,3.267254,0.03008,0.098278,3.237175,1.0,...,0.000000,0.0,0.0,0.0,0.0,9.930953,38.388048,1.239480,15.629609,1
5,0,0,84,110,26,3.267254,0.03008,0.098278,3.237175,1.0,...,0.048779,0.0,0.0,0.0,0.0,14.091701,36.979880,1.732342,15.621549,1
6,0,0,74,95,21,3.267254,0.03008,0.098278,3.237175,1.0,...,0.056748,0.0,0.0,0.0,0.0,10.891867,34.133985,1.789968,16.191501,1
7,0,0,21,59,38,3.267254,0.03008,0.098278,3.237175,1.0,...,0.030275,0.0,0.0,0.0,0.0,4.093972,26.802238,0.806041,12.256102,1
8,0,0,38,79,41,3.267254,0.03008,0.098278,3.237175,1.0,...,0.028947,0.0,0.0,0.0,0.0,4.706527,27.632688,0.961200,13.417604,1
9,0,0,24,52,28,3.267254,0.03008,0.098278,3.237175,1.0,...,0.000000,0.0,0.0,0.0,0.0,2.697300,22.948983,0.702365,11.044126,1


### XGBoost

In [20]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Random Search

In [4]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [5]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [6]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [7]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 100,
 'learning_rate': 0.1,
 'colsample_bytree': 0.6}

In [8]:
preds = xgb_random.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.819828


In [21]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.6,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.808207


In [22]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4,\
                           learning_rate = 0.001, max_depth = 50, n_estimators = 50)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.803082


In [23]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.814766


In [24]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.805560


##### Best params de random forest

In [25]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', learning_rate = 0.1, max_depth = 70, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.812032


In [26]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.7,\
                           learning_rate = 0.1, max_depth = 37, n_estimators = 60)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.811094


In [27]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.807640


# Test

In [17]:
test_set = pd.read_csv('test/test_encoded.csv')

In [18]:
col_id = test_set['id']
test_set.drop('id', axis=1, inplace=True)

In [19]:
preds = xg_reg.predict(test_set)
preds

array([0, 1, 1, ..., 1, 1, 1])

In [20]:
test_set['target'] = preds
test_set['id'] = col_id
test_set[['id', 'target']]

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


### Resultado

In [21]:
test_set[['id', 'target']].to_csv('result.csv', header=True, index=False)